In [ ]:
import numpy as np
import xarray as xr
import xgcm
import seawater as sw

In [ ]:
path = "/path/to/model/output/" 
eddypath = "/path/to/tracked/eddies/"

### Compute vertical heat flux
This is done analogously to the horizontal HTs with $w$, $\langle{w}\rangle$ and $w'$ replacing the total, mean and transient horizontal counterparts, respectively.

First, we define some functions to compute the $'$-quantities.

In [ ]:
def bar_prime(var):
    varbar = var.groupby('time.month').mean("time")
    varprime = (var.groupby('time.month') - varbar)
    return varbar, varprime

In [ ]:
def bar_prime_season(var, season):
    varbar = var.groupby('time.month').mean("time")
    varbar_season = var.groupby('time.season').mean("time").sel(season=season)
    varprime = (var.groupby('time.month') - varbar)
    return varbar_season, varprime

Define constants and load data. We also set up the grid for interpolation.

In [ ]:
rho = 1035.
Cp = 3994.

In [ ]:
data5d = xr.open_zarr(path + "zarr_Diags/output.5d.zarr").sel(time=slice("0201-01-01", "0300-12-30"))

metrics = {
    ('X'): ['dxC', 'dxG', 'dxF', 'dxV'], # X distances
    ('Y'): ['dyC', 'dyG', 'dyF', 'dyU'], # Y distances
    ('Z'): ['drF', 'drW', 'drS', 'drC'], # Z distances
    ('X', 'Y'): ['rAw', 'rAs', 'rA', 'rAz'] # Areas in x-y plane
}

grid = xgcm.Grid(data5d, periodic=["X"], metrics=metrics)

Eddymasks are on F-point due to detection method so need to make sure to interpolate other things to F-point later on!

In [ ]:
eddymask = xr.open_mfdataset(eddypath + 'eddymask_0201-0300.nc', concat_dim="time", combine="nested",
                             data_vars='minimal', coords='minimal', 
                             compat='override').eddymask.squeeze().rename({"lon": "XG", "lat": "YG"})

In [ ]:
eddymask_cyclones = xr.open_mfdataset(eddypath + 'eddymask_cyclones_0201-0300.nc', concat_dim="time", combine="nested",
                                      data_vars='minimal', coords='minimal', 
                                      compat='override').eddymask_cyclones.squeeze().rename({"lon": "XG", "lat": "YG"})

In [ ]:
eddymask_anticyclones = xr.open_mfdataset(eddypath + 'eddymask_anticyclones_0201-0300.nc', concat_dim="time", combine="nested",
                                          data_vars='minimal', coords='minimal', 
                                          compat='override').eddymask_anticyclones.squeeze().rename({"lon": "XG", "lat": "YG"})

In [ ]:
seasons = ["DJF", "MAM", "JJA", "SON"]

Loop over the 10 decades and compute the mean HT for the whole year and the four seasons within each decade.

In [ ]:
for i in range(0, 10):
    print(i)
    # extract the desired decade
    data = data5d.isel(time=slice(720 * i, 720 * (i+1)))
    y1 = str(data.time.isel(time=0).dt.year.values)
    y2 = str(data.time.isel(time=-1).dt.year.values)
    # this is the timestamp used in the saved files later on
    savetime = data.time.isel(time=slice(int(len(data.time) / 2), int(len(data.time) / 2) + 1))
    # compute the reference temperature
    tref = sw.eos80.fp(data.SALT, 1.065)
    # define the mean winter mixed layer depth, the depth across which we compute VHT
    meanJJAMLD = grid.interp(data.MXLDEPTH.groupby("time.season").mean("time").sel(season="JJA"), ("X", "Y"), boundary="extend").compute()
    SIarea = grid.interp(data.SIarea, ("X", "Y"), boundary="extend")
    # need to interpolate THETA to the W-grid
    w = data.WVEL
    TonW = grid.interp(data.THETA - tref, "Z", boundary="extend")
    T = TonW
    # compute mean and anomalies
    wbar, wprime = bar_prime(w)
    Tbar, Tprime = bar_prime(T)
    # compute heat transports (we multiply already by dx because we will integrate in x-direction later)
    # and we interpolate to F-points because that's were the eddies are defined
    VHTtotal = (grid.interp(rho * Cp * w * T, ("Y", "X"), boundary="extend") * data.dxV)
    VHTprime = (grid.interp(rho * Cp * wprime * Tprime, ("Y", "X"), boundary="extend") * data.dxV)
    VHTbar = (grid.interp(rho * Cp * wbar * Tbar, ("Y", "X"), boundary="extend") * data.dxV)
    # contribution by CME
    VHTprime_eddy = VHTprime * eddymask
    VHTprime_eddy_anticyclones = VHTprime * eddymask_anticyclones
    VHTprime_eddy_cyclones = VHTprime * eddymask_cyclones
    ## same for diffusive part
    DIFF = data.DFrI_TH
    DIFFbar, DIFFprime = bar_prime(DIFF)
    VHTDIFFtotal = grid.interp(rho * Cp * DIFF, ("X", "Y"), boundary="extend")
    VHTDIFFprime = grid.interp(rho * Cp * DIFFprime, ("X", "Y"), boundary="extend")
    VHTDIFFbar = grid.interp(rho * Cp * DIFFbar, ("X", "Y"), boundary="extend")
    VHTDIFFprime_eddy = VHTDIFFprime * eddymask
    VHTDIFFprime_eddy_anticyclones = VHTDIFFprime * eddymask_anticyclones
    VHTDIFFprime_eddy_cyclones = VHTDIFFprime * eddymask_cyclones
    # select heat transports at the base of the winter ML 
    VHTtotalMLD = VHTtotal.sel(Zl=-meanJJAMLD, method="ffill")
    VHTprimeMLD = VHTprime.sel(Zl=-meanJJAMLD, method="ffill")
    VHTbarMLD = VHTbar.sel(Zl=-meanJJAMLD, method="ffill")
    VHTprimeMLD_eddy = VHTprime_eddy.sel(Zl=-meanJJAMLD, method="ffill")
    VHTprimeMLD_eddy_cyclones = VHTprime_eddy_cyclones.sel(Zl=-meanJJAMLD, method="ffill")
    VHTprimeMLD_eddy_anticyclones = VHTprime_eddy_anticyclones.sel(Zl=-meanJJAMLD, method="ffill")
    VHTDIFFtotalMLD = VHTDIFFtotal.sel(Zl=-meanJJAMLD, method="ffill")
    VHTDIFFprimeMLD = VHTDIFFprime.sel(Zl=-meanJJAMLD, method="ffill")
    VHTDIFFbarMLD = VHTDIFFbar.sel(Zl=-meanJJAMLD, method="ffill")
    VHTDIFFprimeMLD_eddy = VHTDIFFprime_eddy.sel(Zl=-meanJJAMLD, method="ffill")
    VHTDIFFprimeMLD_eddy_cyclones = VHTDIFFprime_eddy_cyclones.sel(Zl=-meanJJAMLD, method="ffill")
    VHTDIFFprimeMLD_eddy_anticyclones = VHTDIFFprime_eddy_anticyclones.sel(Zl=-meanJJAMLD, method="ffill")
    # zonal integrals (already multiplied by dx above)
    VHTtotalMLDX = VHTtotalMLD.sum("XG")
    VHTprimeMLDX = VHTprimeMLD.sum("XG")
    VHTbarMLDX = VHTbarMLD.sum("XG")
    VHTprimeMLD_eddyX = VHTprimeMLD_eddy.sum("XG")
    VHTprimeMLD_eddy_cyclonesX = VHTprimeMLD_eddy_cyclones.sum("XG")
    VHTprimeMLD_eddy_anticyclonesX = VHTprimeMLD_eddy_anticyclones.sum("XG")
    VHTDIFFtotalMLDX = VHTDIFFtotalMLD.sum("XG")
    VHTDIFFprimeMLDX = VHTDIFFprimeMLD.sum("XG")
    VHTDIFFbarMLDX = VHTDIFFbarMLD.sum("XG")
    VHTDIFFprimeMLD_eddyX = VHTDIFFprimeMLD_eddy.sum("XG")
    VHTDIFFprimeMLD_eddy_cyclonesX = VHTDIFFprimeMLD_eddy_cyclones.sum("XG")
    VHTDIFFprimeMLD_eddy_anticyclonesX = VHTDIFFprimeMLD_eddy_anticyclones.sum("XG")
    # create a dataset and write all the different terms into it (time-averaged over the current decade)
    VHT_across_MLD = xr.Dataset(coords={"time": savetime, "YG": data.YG.values})
    VHT_across_MLD["VHTtotal"] = xr.DataArray(VHTtotalMLDX.mean("time").values[None, :], dims=("time", "YG"))
    VHT_across_MLD["VHTprime"] = xr.DataArray(VHTprimeMLDX.mean("time").values[None, :], dims=("time", "YG"))
    VHT_across_MLD["VHTbar"] = xr.DataArray(VHTbarMLDX.mean("month").values[None, :], dims=("time", "YG"))
    VHT_across_MLD["VHTeddy"] = xr.DataArray(VHTprimeMLD_eddyX.mean("time").values[None, :], dims=("time", "YG"))
    VHT_across_MLD["VHTeddy_cyclones"] = xr.DataArray(VHTprimeMLD_eddy_cyclonesX.mean("time").values[None, :], dims=("time", "YG"))
    VHT_across_MLD["VHTeddy_anticyclones"] = xr.DataArray(VHTprimeMLD_eddy_anticyclonesX.mean("time").values[None, :], dims=("time", "YG"))
    VHT_across_MLD["VHTDIFFtotal"] = xr.DataArray(VHTDIFFtotalMLDX.mean("time").values[None, :], dims=("time", "YG"))
    VHT_across_MLD["VHTDIFFprime"] = xr.DataArray(VHTDIFFprimeMLDX.mean("time").values[None, :], dims=("time", "YG"))
    VHT_across_MLD["VHTDIFFbar"] = xr.DataArray(VHTDIFFbarMLDX.mean("month").values[None, :], dims=("time", "YG"))
    VHT_across_MLD["VHTDIFFeddy"] = xr.DataArray(VHTDIFFprimeMLD_eddyX.mean("time").values[None, :], dims=("time", "YG"))
    VHT_across_MLD["VHTDIFFeddy_cyclones"] = xr.DataArray(VHTDIFFprimeMLD_eddy_cyclonesX.mean("time").values[None, :], dims=("time", "YG"))
    VHT_across_MLD["VHTDIFFeddy_anticyclones"] = xr.DataArray(VHTDIFFprimeMLD_eddy_anticyclonesX.mean("time").values[None, :], dims=("time", "YG"))
    # also add the mixed layer depth used to compute the HT to the dataset
    VHT_across_MLD["MLD"] = meanJJAMLD.mean("XG")
    # write dataset to disk
    VHT_across_MLD.to_netcdf(path + "post/VHT_across_base_of_winter_MLD.new0.3.0" + y1 + "_0" + y2 + ".all.no_int.nc")
    # repeat everything for the different seasons
    for season in seasons:
        print(season)
        wbar, _ = bar_prime_season(w, season)
        Tbar, _ = bar_prime_season(T, season)
        DIFFbar, _ = bar_prime_season(DIFF, season)
        VHTbar = (grid.interp(rho * Cp * wbar * Tbar, ("Y", "X"), boundary="extend") * data.dxV).compute()
        VHTDIFFbar = grid.interp(rho * Cp * DIFFbar, ("X", "Y"), boundary="extend").compute()
        VHTbarMLD = VHTbar.sel(Zl=-meanJJAMLD, method="ffill")
        VHTDIFFbarMLD = VHTDIFFbar.sel(Zl=-meanJJAMLD, method="ffill")
        VHTbarMLDX = VHTbarMLD.sum("XG")
        VHTDIFFbarMLDX = VHTDIFFbarMLD.sum("XG")
        #
        VHT_across_MLD = xr.Dataset(coords={"time": savetime, "YG": data.YG.values})
        VHT_across_MLD["VHTprime"] =\
            xr.DataArray(VHTprimeMLDX.groupby("time.season").mean("time").sel(season=season).values[None, :], dims=("time", "YG"))
        VHT_across_MLD["VHTtotal"] =\
            xr.DataArray(VHTtotalMLDX.groupby("time.season").mean("time").sel(season=season).values[None, :], dims=("time", "YG"))
        VHT_across_MLD["VHTbar"] =\
            xr.DataArray(VHTbarMLDX.values[None, :], dims=("time", "YG"))
        VHT_across_MLD["VHTeddy"] =\
            xr.DataArray(VHTprimeMLD_eddyX.groupby("time.season").mean("time").sel(season=season).values[None, :], dims=("time", "YG"))
        VHT_across_MLD["VHTeddy_cyclones"] =\
            xr.DataArray(VHTprimeMLD_eddy_cyclonesX.groupby("time.season").mean("time").sel(season=season).values[None, :], dims=("time", "YG"))
        VHT_across_MLD["VHTeddy_anticyclones"] =\
            xr.DataArray(VHTprimeMLD_eddy_anticyclonesX.groupby("time.season").mean("time").sel(season=season).values[None, :], dims=("time", "YG"))
        VHT_across_MLD["VHTDIFFprime"] =\
            xr.DataArray(VHTDIFFprimeMLDX.groupby("time.season").mean("time").sel(season=season).values[None, :], dims=("time", "YG"))
        VHT_across_MLD["VHTDIFFbar"] =\
            xr.DataArray(VHTDIFFbarMLDX.values[None, :], dims=("time", "YG"))
        VHT_across_MLD["VHTDIFFeddy"] =\
            xr.DataArray(VHTDIFFprimeMLD_eddyX.groupby("time.season").mean("time").sel(season=season).values[None, :], dims=("time", "YG"))
        VHT_across_MLD["VHTDIFFeddy_cyclones"] =\
            xr.DataArray(VHTDIFFprimeMLD_eddy_cyclonesX.groupby("time.season").mean("time").sel(season=season).values[None, :], dims=("time", "YG"))
        VHT_across_MLD["VHTDIFFeddy_anticyclones"] =\
            xr.DataArray(VHTDIFFprimeMLD_eddy_anticyclonesX.groupby("time.season").mean("time").sel(season=season).values[None, :], dims=("time", "YG"))
        #
        VHT_across_MLD["MLD"] = meanJJAMLD.mean("XG")
        VHT_across_MLD.to_netcdf(path + "post/VHT_across_base_of_winter_MLD." + season + "0" + y1 + "_0" + y2 + ".nc")